### **trustyai.ipynb**
### **Demo of implementing Explainable AI (XAI) using TrustyAI**

* ##### 01 - Setup
* ##### 02 - Install packages
* ##### 03 - Import packages
* ##### 04 - Load dataset
* ##### 05 - Prepare dataset
* ##### 06 - Create model
* ##### 07 - Run predictions
* ##### 08 - Create TrustyAI model
* ##### 09 - Local Interpretable Model-Agnostic Explanations (LIME)
* ##### 10 - Shapley Additive Explanations (SHAP)
* ##### 11 - Tyrus

### 01 - Setup

In [ ]:
from os  import environ, getenv
from sys import executable

In [ ]:
if not getenv('JAVA_HOME'):

    from subprocess import check_call
    check_call([executable, '-m', 'pip', 'install', '--upgrade', 'install-jdk'])

    from jdk import install
    environ['JAVA_HOME'] = install('17')

### 02 - Install packages

In [ ]:
!{ executable } -m pip install --upgrade bokeh pandas scikit-learn trustyai xgboost

### 03 - Import packages

In [ ]:
from bokeh.io                import output_notebook
from pandas                  import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import LabelEncoder
from trustyai.explainers     import LimeExplainer, SHAPExplainer
from trustyai.model          import Model
from trustyai.utils.tyrus    import Tyrus
from xgboost                 import XGBClassifier

### 04 - Load dataset

In [ ]:
dataset = read_csv('../datasets/loan.csv')

In [ ]:
dataset.head()

### 05 - Prepare dataset

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
features = dataset.drop('Loan_ID', axis = 1).drop('Loan_Status', axis = 1)
features = features.apply(label_encoder.fit_transform)

In [ ]:
labels = dataset[['Loan_Status']]
labels = labels.apply(label_encoder.fit_transform)

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels)

### 06 - Create model

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(features_train, labels_train)

In [ ]:
model.score(features_test, labels_test)

### 07 - Run predictions

In [ ]:
example_0 = features_test.iloc[0]

In [ ]:
example_0

In [ ]:
example_1 = features_test.iloc[1]

In [ ]:
example_1

In [ ]:
predictions = model.predict([example_0, example_1])

In [ ]:
predictions

### 08 - Create TrustyAI model

In [ ]:
trustyai_model = Model(
    fn              = model.predict,
    feature_names   = list(features_train),
    output_names    = list(labels_train),
    dataframe_input = True
)

### 09 - Local Interpretable Model-Agnostic Explanations (LIME)

In [ ]:
lime_explainer = LimeExplainer()

In [ ]:
lime_explanation_0 = lime_explainer.explain(
    model   = trustyai_model,
    inputs  = example_0.astype(float),
    outputs = predictions[0]
)

In [ ]:
lime_explanation_0.as_html()['Loan_Status']

In [ ]:
lime_explanation_1 = lime_explainer.explain(
    model   = trustyai_model,
    inputs  = example_1.astype(float),
    outputs = predictions[1]
)

In [ ]:
lime_explanation_1.as_html()['Loan_Status']

### 10 - Shapley Additive Explanations (SHAP)

In [ ]:
shap_explainer = SHAPExplainer(background = features_train[:100].astype(float))

In [ ]:
shap_explanation_0 = shap_explainer.explain(
    model   = trustyai_model,
    inputs  = example_0.astype(float),
    outputs = predictions[0]
)

In [ ]:
shap_explanation_0.as_html()['Loan_Status']

In [ ]:
shap_explanation_1 = shap_explainer.explain(
    model   = trustyai_model,
    inputs  = example_1.astype(float),
    outputs = predictions[1]
)

In [ ]:
shap_explanation_1.as_html()['Loan_Status']

### 11 - Tyrus

In [ ]:
tyrus_0 = Tyrus(
    background = features_train[:100].astype(float),
    model      = trustyai_model,
    inputs     = example_0.astype(float),
    outputs    = predictions[0]
)

In [ ]:
output_notebook()

In [ ]:
tyrus_0.run()

In [ ]:
tyrus_1 = Tyrus(
    background = features_train[:100].astype(float),
    model      = trustyai_model,
    inputs     = example_1.astype(float),
    outputs    = predictions[1]
)

In [ ]:
output_notebook()

In [ ]:
tyrus_1.run()